## Experimento
Neste notebook, serão apresentados o código utilizado no experimento

### Importação das bibliotecas

In [1]:
import time
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
import joblib  # Para salvar e carregar o modelo
from sklearn.svm import SVC
from sklearn.model_selection import KFold 


#importando bibliotecas
from tensorflow import keras
from sklearn.utils import shuffle
#from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM,Bidirectional,Dropout
from keras.layers import AveragePooling1D
from keras.models import load_model
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam, RMSprop
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.layers import Flatten
from sklearn import preprocessing
from sklearn.metrics import * 
import os
from imblearn.over_sampling import SMOTE #normalizacao da base
#Bibliotecas para rodar os algoritmos em modo classificação
from collections import Counter
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import time
from sklearn.utils.class_weight import compute_class_weight



2024-12-02 19:33:37.639673: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-02 19:33:37.641021: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-02 19:33:37.647663: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-02 19:33:37.668032: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-02 19:33:37.700695: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

### Carregamento dos dados

In [2]:
X_embedding = pd.read_csv("output_embeddings_xlnet_pt.csv", header=None) #Xlnet

In [3]:
y = pd.read_csv('estiDeep.data', header=None) #File containing the set of training and test labels.
y = np.array(y)
print ('Shape of label tensor:', y.shape)
print(y.dtype)

Shape of label tensor: (23313, 1)
int64


In [4]:
#Realizando o mapeamento para a sequencia de fibonacci
fibonacci_labels = [1, 2, 3, 5, 8, 13, 20, 40, 100]
#arredondando para o valor mais próximo dos rótulos Fibonacci.
def map_to_fibonacci(estimate):
    if estimate <= 1.5:
        return 1
    elif estimate <= 2.5:
        return 2
    elif estimate <= 4:
        return 3
    elif estimate <= 6.5:
        return 5
    elif estimate <= 10.5:
        return 8
    elif estimate <= 16.5:
        return 13
    elif estimate <= 30:
        return 20
    elif estimate <= 70:
        return 40
    else:
        return 100



In [7]:
map_to_fibonacci_vec = np.vectorize(map_to_fibonacci)
fibonacci_labels = map_to_fibonacci_vec(y)
y_fibonacci = fibonacci_labels

In [8]:
y_fibonacci_values = y_fibonacci.flatten()
y_fibonacci_values.shape

(23313,)

### Realizando Normalização da Base de Dados com SMOTE

In [11]:
df_y_labels = pd.DataFrame(y_fibonacci_values, columns=["Pontos"])


In [13]:

#Contar a quantidade de itens em cada classe
class_counts = pd.Series(y_fibonacci_values).value_counts().sort_index()

#Exibir a quantidade de itens por classe
print("Quantidade de itens por classe:")
for classe, quantidade in class_counts.items():
    print(f"Classe {classe}: {quantidade} itens")





Quantidade de itens por classe:
Classe 1: 4225 itens
Classe 2: 3406 itens
Classe 3: 4809 itens
Classe 5: 4725 itens
Classe 8: 3588 itens
Classe 13: 1238 itens
Classe 20: 764 itens
Classe 40: 397 itens
Classe 100: 161 itens


In [14]:
#Verificando shape x e y
print("shape X:", X_embedding.shape) #vetores embeddings
print("shape Y:", y_fibonacci.shape)

shape X: (23313, 768)
shape Y: (23313, 1)


In [15]:
# Aplicar o SMOTE
smote = SMOTE(sampling_strategy='auto')
X_res, y_res = smote.fit_resample(X_embedding, y_fibonacci)

In [16]:
#Numero de amostras apos resample"
print("shape X:", X_res.shape) #vetores embeddings
print("shape Y:", y_res.shape)

shape X: (43281, 768)
shape Y: (43281,)


In [17]:
# Contar a quantidade de itens em cada classe
class_counts = pd.Series(y_res).value_counts().sort_index()

# Exibir a quantidade de itens por classe
print("Quantidade de itens por classe:")
for classe, quantidade in class_counts.items():
    print(f"Classe {classe}: {quantidade} itens")

Quantidade de itens por classe:
Classe 1: 4809 itens
Classe 2: 4809 itens
Classe 3: 4809 itens
Classe 5: 4809 itens
Classe 8: 4809 itens
Classe 13: 4809 itens
Classe 20: 4809 itens
Classe 40: 4809 itens
Classe 100: 4809 itens


### Ajustando o formato de dados para treinamento e teste

In [18]:
#tornando X um array numpy
X_res_values = X_res.values

# Defina o número de sentenças
num_sentences = 115
num_samples = X_res_values.shape[0]
embedding_dim = X_res_values.shape[1]

# Truncar e reformular os dados para a entrada do modelo
needed_samples = (num_samples // num_sentences) * num_sentences
X_truncated = X_res_values[:needed_samples]
y_truncated = y_res[:needed_samples]

# Reformulando X para o formato adequado para o LSTM
X_reshaped = X_truncated.reshape(-1, num_sentences, embedding_dim)
y_truncated_resized = y_truncated[:X_reshaped.shape[0]]
# Inicialização para armazenar as métricas
all_y_test = []
all_y_pred = []

# Definir número de classes para a camada Softmax
num_classes = 9  # Correspondente a 9 classes (0-8)

#definindo variavel para realizar cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=1000)
#kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1000)

In [19]:
print(X_reshaped.shape)
print(y_truncated.shape)
print(X_truncated.shape)

(376, 115, 768)
(43240,)
(43240, 768)


In [20]:
#convertendo as classes originais em 0,1,2,3,4,5,6,7,8
# Classes originais
original_classes = [1, 2, 3, 5, 8, 13, 20, 40, 100]
class_to_index = {cls: idx for idx, cls in enumerate(original_classes)}
index_to_class = {idx: cls for idx, cls in enumerate(original_classes)}

y_encoded = np.array([class_to_index[y] for y in y_truncated])

In [ ]:
np.unique(y_encoded)

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

### Modelo Deep Learning
Este modelo utiliza a mesma arquitetura que foi utilizada no modelo de regressão, porém com as alterações:

1- Remoção da camada de embeddings (para comparar com RF e SVC e devido os dados ja estarem convertidos em vetores embeddings com xlnet)

2- Alteração da camada linear pela camada de softmax

3- função de perda:sparse_categorical_crossentropy

O restante segue da mesma forma que o modelo de regressão. Lembrando que o objetivo era utilizar o mesmo modelo deep learning para a tarefa de classificação.

In [23]:


# Inicializando listas para armazenar métricas e tempos
vetAccuracyDeep = []
vetPrecisionDeep = []
vetRecallDeep = []
vetF1Deep = []
training_times = []
inference_times = []
all_y_test_Deep = []
all_y_pred_Deep = []

for train_index, test_index in kf.split(X_truncated):
    print(X_truncated.shape)
    print(X_truncated.size)   
    x_train, x_test = X_truncated[train_index], X_truncated[test_index]
    y_train, y_test = y_encoded[train_index], y_encoded[test_index]
    print(np.unique(y_train))
    print(np.unique(y_test))
    print(x_train.shape)  #para x_train
    print(x_test.shape)   #Para x_test
    n_sentence = 1

    x_train = x_train.reshape(x_train.shape[0], n_sentence, x_train.shape[1])  # (38947, 1, 768)
    x_test = x_test.reshape(x_test.shape[0], n_sentence, x_test.shape[1])  # (4328, 1, 768)

    #Definindo o modelo

    modelDeepClass = Sequential([
        LSTM(100, dropout=0.5, recurrent_dropout=0.3, return_sequences=False),
        Dense(50, activation='relu'),
        Dense(10, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])

    adam = Adam(learning_rate=0.001)
    modelDeepClass.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

    # Early stopping
    es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=50, restore_best_weights=True)

    # Treinando o modelo
    start_training = time.time()
    modelDeepClass.fit(
        x_train, y_train,
        batch_size=128,
        epochs=30,
        validation_data=(x_test, y_test),
        callbacks=[es],
        verbose=0
    )
    end_training = time.time()
    training_time = end_training - start_training
    training_times.append(training_time)
    # Fazendo previsões e medindo o tempo
    start_inference = time.time()
    y_pred = modelDeepClass.predict(x_test)
    end_inference = time.time()
    inference_time = end_inference - start_inference
    inference_times.append(inference_time)

    y_pred_classes = np.argmax(y_pred, axis=1)

    # Calculando métricas
    accuracy = accuracy_score(y_test, y_pred_classes)
    class_report = classification_report(y_test, y_pred_classes, output_dict=True)

    vetAccuracyDeep.append(accuracy)
    vetPrecisionDeep.append(class_report['macro avg']['precision'])
    vetRecallDeep.append(class_report['macro avg']['recall'])
    vetF1Deep.append(class_report['macro avg']['f1-score'])

    all_y_test_Deep.extend(y_test)
    all_y_pred_Deep.extend(y_pred_classes)

# Relatório final de métricas
print(f"Média Accuracy: {np.mean(vetAccuracyDeep):.4f} ± {np.std(vetAccuracyDeep):.4f}")
print(f"Média Precision: {np.mean(vetPrecisionDeep):.4f} ± {np.std(vetPrecisionDeep):.4f}")
print(f"Média Recall: {np.mean(vetRecallDeep):.4f} ± {np.std(vetRecallDeep):.4f}")
print(f"Média F1-Score: {np.mean(vetF1Deep):.4f} ± {np.std(vetF1Deep):.4f}")
print(f"Tempo médio de treinamento: {np.mean(training_times):.2f} ± {np.std(training_times):.2f} segundos")
print(f"Tempo médio de inferência: {np.mean(inference_times):.2f} ± {np.std(inference_times):.2f} segundos")


(43240, 768)
33208320
[0 1 2 3 4 5 6 7 8]
[0 1 2 3 4 5 6 7 8]
(38916, 768)
(4324, 768)


KeyboardInterrupt: 

In [ ]:
# Relatório de métricas finais
print(f'Média Accuracy: {np.mean(vetAccuracyDeep)} ± {np.std(vetAccuracyDeep)}')
print(f'Média Precision: {np.mean(vetPrecisionDeep)} ± {np.std(vetPrecisionDeep)}')
print(f'Média Recall: {np.mean(vetRecallDeep)} ± {np.std(vetRecallDeep)}')
print(f'Média F1-Score: {np.mean(vetF1Deep)} ± {np.std(vetF1Deep)}')

In [ ]:


# Definir as classes do experimento E5


# Converter as previsões para classes discretas

#all_y_test_Deep, all_y_pred_Deep,
#classes = np.array([1, 2, 3, 5, 8, 13, 20, 40, 100])
classes = np.array([1, 2, 3, 4, 5, 6, 7, 8])

# Gerar a matriz de confusão
matriz_confusao_deep = confusion_matrix(all_y_test_Deep, all_y_pred_Deep, labels=classes)

# Calcular a proporção para cada classe verdadeira (linha)
matriz_confusao_proporcao = matriz_confusao.astype('float') / matriz_confusao_deep.sum(axis=1)[:, np.newaxis]

# Exibir a matriz de confusão usando seaborn para visualização
plt.figure(figsize=(10, 8))
sns.heatmap(matriz_confusao_proporcao, annot=True, fmt=".2f", cmap="Blues", xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix - Deep Learning Model (Proportion)')
plt.show()

### Random Forest

In [ ]:

# Inicializando listas para métricas e tempos
vetAccuracyRF = []
vetPrecisionRF = []
vetRecallRF = []
vetF1RF = []
training_times_rf = []
inference_times_rf = []

all_y_test_RF = []
all_y_pred_RF = []


# Inicializando validação cruzada estratificada
#kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Loop pelos folds
for train_index, test_index in kf.split(X_truncated):
    x_train, x_test = X_truncated[train_index], X_truncated[test_index]
    y_train, y_test = y_encoded[train_index], y_encoded[test_index]
    
    # Verificando balanceamento das classes
    print("Distribuição de classes no treino:", Counter(y_train))
    print("Distribuição de classes no teste:", Counter(y_test))

    # Instanciando o modelo Random Forest
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    
    # Treinando o modelo e medindo o tempo de treinamento
    start_training = time.time()
    rf_model.fit(x_train, y_train)
    end_training = time.time()
    training_time = end_training - start_training
    training_times_rf.append(training_time)
    
    # Fazendo previsões e medindo o tempo de inferência
    start_inference = time.time()
    y_pred = rf_model.predict(x_test)
    end_inference = time.time()
    inference_time = end_inference - start_inference
    inference_times_rf.append(inference_time)
    
    # Calculando métricas
    accuracy = accuracy_score(y_test, y_pred)
    class_report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    
    vetAccuracyRF.append(accuracy)
    vetPrecisionRF.append(class_report['macro avg']['precision'])
    vetRecallRF.append(class_report['macro avg']['recall'])
    vetF1RF.append(class_report['macro avg']['f1-score'])
    
    all_y_test_RF.extend(y_test)
    all_y_pred_RF.extend(y_pred)

# Salvando o modelo treinado com joblib
model_filename = f"random_forest_.joblib"
joblib.dump(rf_model, model_filename)
print(f"Modelo salvo em {model_filename}")

# Relatório final de métricas
print(f"Média Accuracy: {np.mean(vetAccuracyRF):.4f} ± {np.std(vetAccuracyRF):.4f}")
print(f"Média Precision: {np.mean(vetPrecisionRF):.4f} ± {np.std(vetPrecisionRF):.4f}")
print(f"Média Recall: {np.mean(vetRecallRF):.4f} ± {np.std(vetRecallRF):.4f}")
print(f"Média F1-Score: {np.mean(vetF1RF):.4f} ± {np.std(vetF1RF):.4f}")
print(f"Tempo médio de treinamento: {np.mean(training_times_rf):.2f} ± {np.std(training_times_rf):.2f} segundos")
print(f"Tempo médio de inferência: {np.mean(inference_times_rf):.2f} ± {np.std(inference_times_rf):.2f} segundos")


In [ ]:


# Definir as classes do experimento E5


# Converter as previsões para classes discretas

#all_y_test_Deep, all_y_pred_Deep,
#classes = np.array([1, 2, 3, 5, 8, 13, 20, 40, 100])
classes = np.array([1, 2, 3, 4, 5, 6, 7, 8])

# Gerar a matriz de confusão
matriz_confusao_rf = confusion_matrix(all_y_test_RF, all_y_pred_RF, labels=classes)

# Calcular a proporção para cada classe verdadeira (linha)
matriz_confusao_proporcao_rf = matriz_confusao_rf.astype('float') / matriz_confusao_rf.sum(axis=1)[:, np.newaxis]

# Exibir a matriz de confusão usando seaborn para visualização
plt.figure(figsize=(10, 8))
sns.heatmap(matriz_confusao_proporcao_rf, annot=True, fmt=".2f", cmap="Blues", xticklabels=classes, yticklabels=classes)
plt.xlabel('Rótulo predito')
plt.ylabel('Rótulo verdadeiro')
plt.title('Matriz de Confusão - Random Forest (pt)')
plt.show()

### SCV

In [ ]:

#Inicializando listas para métricas e tempos
vetAccuracySVM = []
vetPrecisionSVM = []
vetRecallSVM = []
vetF1SVM = []
training_times_svm = []
inference_times_svm = []

all_y_test_svm = []
all_y_pred_svm= []


# Loop pelos folds
for train_index, test_index in kf.split(X_truncated):
    x_train, x_test = X_truncated[train_index], X_truncated[test_index]
    y_train, y_test = y_encoded[train_index], y_encoded[test_index]
    
    # Verificando balanceamento das classes
    print("Distribuição de classes no treino:", Counter(y_train))
    print("Distribuição de classes no teste:", Counter(y_test))

    #Instanciando o modelo SVM
    svm_model = SVC(kernel='rbf', random_state=42)

    #Treinando o modelo e medindo o tempo de treinamento
    start_training_svm = time.time()
    svm_model.fit(x_train, y_train)
    end_training_svm = time.time()
    training_time_svm = end_training_svm - start_training_svm
    training_times_svm.append(training_time_svm)
    
    #Fazendo previsões e medindo o tempo de inferência
    start_inference = time.time()
    y_pred = svm_model.predict(x_test)
    end_inference = time.time()
    inference_time = end_inference - start_inference
    inference_times_svm.append(inference_time)
    
    #Calculando métricas
    accuracy = accuracy_score(y_test, y_pred)
    class_report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    
    vetAccuracySVM.append(accuracy)
    vetPrecisionSVM.append(class_report['macro avg']['precision'])
    vetRecallSVM.append(class_report['macro avg']['recall'])
    vetF1SVM.append(class_report['macro avg']['f1-score'])
    
    all_y_test_svm.extend(y_test)
    all_y_pred_svm.extend(y_pred)

#Salvando o modelo treinado com joblib
model_filename = f"svm_model.joblib"
joblib.dump(svm_model, model_filename)
print(f"Modelo salvo em {model_filename}")

#Relatório final de métricas
print(f"Média Accuracy: {np.mean(vetAccuracySVM):.4f} ± {np.std(vetAccuracySVM):.4f}")
print(f"Média Precision: {np.mean(vetPrecisionSVM):.4f} ± {np.std(vetPrecisionSVM):.4f}")
print(f"Média Recall: {np.mean(vetRecallSVM):.4f} ± {np.std(vetRecallSVM):.4f}")
print(f"Média F1-Score: {np.mean(vetF1SVM):.4f} ± {np.std(vetF1SVM):.4f}")
print(f"Tempo médio de treinamento: {np.mean(training_times_svm):.2f} ± {np.std(training_times_svm):.2f} segundos")
print(f"Tempo médio de inferência: {np.mean(inference_times_svm):.2f} ± {np.std(inference_times_svm):.2f} segundos")




In [ ]:


# Definir as classes do experimento E5


# Converter as previsões para classes discretas

#all_y_test_Deep, all_y_pred_Deep,
#classes = np.array([1, 2, 3, 5, 8, 13, 20, 40, 100])
classes = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8])

# Gerar a matriz de confusão
matriz_confusao = confusion_matrix(all_y_test_svm, all_y_pred_svm, labels=classes)

# Calcular a proporção para cada classe verdadeira (linha)
matriz_confusao_proporcao = matriz_confusao.astype('float') / matriz_confusao.sum(axis=1)[:, np.newaxis]

# Exibir a matriz de confusão usando seaborn para visualização
plt.figure(figsize=(10, 8))
sns.heatmap(matriz_confusao_proporcao, annot=True, fmt=".2f", cmap="Blues", xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix - SVM Model (Proportion)')
plt.show()